## Required Library

In [ ]:
import cv2 
import tifffile 
import numpy as np
import math
import pandas as pd
import json

import skimage 
import skimage.io
import skimage.measure
import skimage.morphology
from scipy.io import loadmat

import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify visible gpu(s) 0-3
import tensorflow as tf
try:
    tf_gpus = tf.config.list_physical_devices('GPU')
    for gpu in tf_gpus:
        tf.config.experimental.set_memory_growth(gpu, True) # allow memory growth on visible gpu(s)
except:
    pass 

import skimage.io as io
from datetime import datetime
from matplotlib import pyplot as plt
date = datetime.today().strftime('%Y-%m-%d')

from deepcell.applications import Mesmer
from deepcell.utils.plot_utils import create_rgb_image, make_outline_overlay

from INDEPTH_functions import process_fusion_image, segment_image

from tqdm import tqdm
import shutil


## Fusion scan

### Load Fusion scan

Load the 8-bit qptiffs.

In [ ]:
DFCI_scan = tifffile.imread('../data/INDEPTH_DLBCL_Right_Scan1.qptiff')

Rochester_scan = tifffile.imread('../data/INDEPTH_DLBCL_Left_Scan1.qptiff')

### Load core position csv

Load the csv that record the position of each core.

In [ ]:
DFCI_core_position = pd.read_csv('../data/DFCI_core_position.csv')
Rochester_core_position = pd.read_csv('../data/Rochester_core_position.csv')

Create an empty canvas for putting the cores back after crop.

In [ ]:
DFCI_canvas = np.zeros_like(DFCI_scan[0], 'uint8')
Rochester_canvas = np.zeros_like(Rochester_scan[0], 'uint8')

Crop cores and put them back into the empty canvas.

In [ ]:
for i in DFCI_core_position['Core']:
    x1 = DFCI_core_position[DFCI_core_position['Core'] == i]['x1'].values[0]
    x2 = DFCI_core_position[DFCI_core_position['Core'] == i]['x2'].values[0]
    y1 = DFCI_core_position[DFCI_core_position['Core'] == i]['y1'].values[0]
    y2 = DFCI_core_position[DFCI_core_position['Core'] == i]['y2'].values[0]
    core_img = DFCI_scan[0][y1:y2, x1:x2]
    #tifffile.imshow(core_img)
    DFCI_canvas[y1:y2, x1:x2] = core_img


for i in Rochester_core_position['Core']:
    x1 = Rochester_core_position[Rochester_core_position['Core'] == i]['x1'].values[0]
    x2 = Rochester_core_position[Rochester_core_position['Core'] == i]['x2'].values[0]
    y1 = Rochester_core_position[Rochester_core_position['Core'] == i]['y1'].values[0]
    y2 = Rochester_core_position[Rochester_core_position['Core'] == i]['y2'].values[0]
    core_img = Rochester_scan[0][y1:y2, x1:x2]
    #tifffile.imshow(core_img)
    Rochester_canvas[y1:y2, x1:x2] = core_img




Visual check to make sure the coordinates are correct 

In [ ]:
fig= plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(DFCI_scan[0], cmap='gray')
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(DFCI_canvas, cmap = 'gray', alpha=1)
plt.tight_layout()
plt.show()


In [ ]:
fig= plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(Rochester_scan[0], cmap='gray')
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(Rochester_canvas, cmap = 'gray', alpha=1)
plt.tight_layout()
plt.show()

### Rotate Fusion scan

Since later we will be registering the Fusion scan and the GeoMx scan. We will now rotate the Fusion scan clockwise 180 degrees to have them in the same orientation as the GeoMx scan, in order to eliminate one variable when registering.

In [ ]:
DFCI_rotate_list = []
Rochester_rotate_list = []

for i in range(DFCI_scan.shape[0]):
    print(f'Rotating the {i}th channel')
    channel_rotate = cv2.rotate(DFCI_scan[i], cv2.ROTATE_180)
    DFCI_rotate_list.append(channel_rotate)

DFCI_rotate = np.stack(DFCI_rotate_list)

for i in range(Rochester_scan.shape[0]):
    print(f'Rotating the {i}th channel')
    channel_rotate = cv2.rotate(Rochester_scan[i], cv2.ROTATE_180)
    Rochester_rotate_list.append(channel_rotate)

Rochester_rotate = np.stack(Rochester_rotate_list)


In [ ]:
fig = plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(DFCI_rotate[0], cmap = 'gray')
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(Rochester_rotate[0], cmap = 'gray')
plt.tight_layout()
plt.show()

### Calculate rotated coordinates

In [ ]:
DFCI_core_position['x1_rotate'] = DFCI_scan.shape[2] - DFCI_core_position['x1'] 
DFCI_core_position['x2_rotate'] = DFCI_scan.shape[2] - DFCI_core_position['x2'] 
DFCI_core_position['y1_rotate'] = DFCI_scan.shape[1] - DFCI_core_position['y1'] 
DFCI_core_position['y2_rotate'] = DFCI_scan.shape[1] - DFCI_core_position['y2'] 

In [ ]:
Rochester_core_position['x1_rotate'] = Rochester_scan.shape[2] - Rochester_core_position['x1'] 
Rochester_core_position['x2_rotate'] = Rochester_scan.shape[2] - Rochester_core_position['x2'] 
Rochester_core_position['y1_rotate'] = Rochester_scan.shape[1] - Rochester_core_position['y1'] 
Rochester_core_position['y2_rotate'] = Rochester_scan.shape[1] - Rochester_core_position['y2'] 

### Put the cropped cell back into a blank canvas

Optional step to check if the cores were all rotated correctly and if the calculated rotated coordinates were correct. The DAPI channel were used for visual checking. 

In [ ]:
DFCI_piece_together = np.zeros_like(DFCI_rotate[0], 'uint8')
Rochester_piece_together = np.zeros_like(Rochester_rotate[0], 'uint8')

for i in DFCI_core_position['Core']:
    x1 = DFCI_core_position[DFCI_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = DFCI_core_position[DFCI_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = DFCI_core_position[DFCI_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = DFCI_core_position[DFCI_core_position['Core'] == i]['y1_rotate'].values[0]
    core_img = DFCI_rotate[0][y1:y2, x1:x2]
    #tifffile.imshow(core_img)
    DFCI_piece_together[y1:y2, x1:x2] = core_img

for i in Rochester_core_position['Core']:
    x1 = Rochester_core_position[Rochester_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = Rochester_core_position[Rochester_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = Rochester_core_position[Rochester_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = Rochester_core_position[Rochester_core_position['Core'] == i]['y1_rotate'].values[0]
    core_img = Rochester_rotate[0][y1:y2, x1:x2]
    #tifffile.imshow(core_img)
    Rochester_piece_together[y1:y2, x1:x2] = core_img

In [ ]:
fig = plt.figure(figsize = (12,12))
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(DFCI_rotate[0], cmap = 'gray')
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(Rochester_rotate[0], cmap = 'gray')
ax3 = fig.add_subplot(2,2,3)
ax3.imshow(DFCI_piece_together, cmap = 'gray')
ax4 = fig.add_subplot(2,2,4)
ax4.imshow(Rochester_piece_together, cmap = 'gray')
plt.tight_layout()
plt.show()

### Check json

Check the configuration json files which contains the lower bound for each marker in each core. 

In [ ]:
with open('../data/DFCI_config.json') as json_file:
    data = json.load(json_file)
for key, core_dict in data['cores'].items():
    core_name = core_dict['core_name']
    print(core_name)
    print(DFCI_core_position[DFCI_core_position['Core'] == core_name])

### Process the TMA based on user specified channel and threshold and segment

Processing by setting all signal less than or equal to the lower bound to 0.

In [ ]:
Rochester_core_dict = {}
for i in Rochester_core_position['Core']:
    x1 = Rochester_core_position[Rochester_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = Rochester_core_position[Rochester_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = Rochester_core_position[Rochester_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = Rochester_core_position[Rochester_core_position['Core'] == i]['y1_rotate'].values[0]
    core_img = Rochester_rotate[:, y1:y2, x1:x2]
    print(f"Processing core {i}")
    img_dict = process_fusion_image(core_img, '../data/MarkerList.txt', '../data/Rochester_config.json', coreName= i)
    Rochester_core_dict[i] = img_dict
    
DFCI_core_dict = {}
for i in DFCI_core_position['Core']:
    x1 = DFCI_core_position[DFCI_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = DFCI_core_position[DFCI_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = DFCI_core_position[DFCI_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = DFCI_core_position[DFCI_core_position['Core'] == i]['y1_rotate'].values[0]
    core_img = DFCI_rotate[:, y1:y2, x1:x2]
    print(f"Processing core {i}")
    img_dict = process_fusion_image(core_img, '../data/MarkerList.txt', '../data/DFCI_config.json', coreName= i)
    DFCI_core_dict[i] = img_dict

Segment the cores.

In [ ]:
for i in Rochester_core_position['Core']:
    img_dict = Rochester_core_dict[i]
    segment_image(img_dict, ['DAPI'], ['HLA1', 'HLA-DR', 'CD31'], 0.5, f"../output/seg_results/Rochester/core_{i}/", maxima_threshold=0.075, interior_threshold=0.05, search_mode=False, run_mode=True)
    
for i in DFCI_core_position['Core']:
    img_dict = DFCI_core_dict[i]
    segment_image(img_dict, ['DAPI'], ['HLA1', 'HLA-DR', 'CD31'], 0.5, f"../output/seg_results/DFCI/core_{i}/", maxima_threshold=0.075, interior_threshold=0.05, search_mode=False, run_mode=True)
    

### Put the segmentation mask back for QC

In [ ]:
empty_canvas = np.zeros((Rochester_rotate[0].shape[0], Rochester_rotate[0].shape[1]), 'uint')


for i in Rochester_core_position['Core']:
    x1 = Rochester_core_position[Rochester_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = Rochester_core_position[Rochester_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = Rochester_core_position[Rochester_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = Rochester_core_position[Rochester_core_position['Core'] == i]['y1_rotate'].values[0]
    core_seg = tifffile.imread(f'../output/seg_results/Rochester/core_{i}/MESMER_mask.tiff')
    #tifffile.imshow(core_img)
    empty_canvas[y1:y2, x1:x2] = core_seg


tifffile.imwrite('../data/Rochester_MESMER_mask.tiff', empty_canvas, photometric='minisblack')

### Overall QC

In [ ]:
fig = plt.figure(figsize = (12,12))
ax4 = fig.add_subplot(1,4,1)
ax4.imshow(Rochester_scan[0], cmap = 'gray')
ax4.title.set_text('DAPI')
ax1 = fig.add_subplot(1,4,2)
ax1.imshow(Rochester_rotate[0], cmap = 'gray')
ax1.title.set_text('DAPI rotated')
ax2 = fig.add_subplot(1,4,3)
ax2.imshow(Rochester_piece_together, cmap = 'gray')
ax2.title.set_text('Cores pieced together')
ax3 = fig.add_subplot(1,4,4)
ax3.imshow(empty_canvas, cmap = 'gray')
ax3.title.set_text('Segmentation pieced together')
plt.tight_layout()
plt.show()

In [ ]:
empty_canvas = np.zeros((DFCI_rotate[0].shape[0], DFCI_rotate[0].shape[1]), 'uint')


for i in DFCI_core_position['Core']:
    x1 = DFCI_core_position[DFCI_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = DFCI_core_position[DFCI_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = DFCI_core_position[DFCI_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = DFCI_core_position[DFCI_core_position['Core'] == i]['y1_rotate'].values[0]
    core_seg = tifffile.imread(f'../output/seg_results/DFCI/core_{i}/MESMER_mask.tiff')
    #tifffile.imshow(core_img)
    empty_canvas[y1:y2, x1:x2] = core_seg


tifffile.imwrite('../data/DFCI_MESMER_mask.tiff', empty_canvas, photometric='minisblack')


In [ ]:
fig = plt.figure(figsize = (12,12))
ax4 = fig.add_subplot(1,4,1)
ax4.imshow(DFCI_scan[0], cmap = 'gray')
ax4.title.set_text('DAPI')
ax1 = fig.add_subplot(1,4,2)
ax1.imshow(DFCI_rotate[0], cmap = 'gray')
ax1.title.set_text('DAPI rotated')
ax2 = fig.add_subplot(1,4,3)
ax2.imshow(DFCI_piece_together, cmap = 'gray')
ax2.title.set_text('Cores pieced together')
ax3 = fig.add_subplot(1,4,4)
ax3.imshow(empty_canvas, cmap = 'gray')
ax3.title.set_text('Segmentation pieced together')
plt.tight_layout()
plt.show()

## Extract single cell information from Fusion scan

### Read marker list

In [ ]:
txt_file = open('../data/MarkerList.txt', 'r')

clusterChannels = txt_file.read().split('\n')

txt_file.close()

print(clusterChannels)

### Extract from TMA

Extract single cell information with the segmentation mask.

In [ ]:
for core in Rochester_core_position['Core']:
    array_list = []
    core_img = Rochester_core_dict[core]
    channelNum = len(core_img)
    for channel in clusterChannels:
        m = core_img[channel]
        array_list.append(m)
    countsNoNoise=np.stack(array_list, axis=2) # count matrices in the image

    # load mask
    segMat = skimage.io.imread(f"../output/seg_results/Rochester/core_{core}/MESMER_mask.tiff")
    stats = skimage.measure.regionprops(segMat)
    labelNum = len(stats) # number of actual cells not always equal to np.max(segMat)

    # init empty containers
    data = np.zeros((labelNum,channelNum))
    dataScaleSize = np.zeros((labelNum,channelNum))
    cellSizes = np.zeros((labelNum,1))
    cell_props = np.zeros((labelNum, 3))

    # extract info
    for i in range(labelNum): # for each cell (label)
        cellLabel = stats[i].label
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i, 0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes
        cell_props[i, 0] = cellLabel
        cell_props[i, 1] = stats[i].centroid[0] # Y_cent
        cell_props[i, 2] = stats[i].centroid[1] # X_cent

    data_df = pd.DataFrame(data)
    data_df.columns = clusterChannels
    data_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), data_df), axis=1)

    dataScaleSize_df = pd.DataFrame(dataScaleSize)
    dataScaleSize_df.columns = clusterChannels
    dataScaleSize_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), dataScaleSize_df), axis = 1)

    # save all dfs
    save_dir = f'../output/extracted_info_052024/Rochester/core_{core}'

    if os.path.exists(save_dir) == False:
        os.makedirs(save_dir)

    data_full.to_csv(save_dir + "/data.csv", index = False)
    dataScaleSize_full.to_csv(save_dir + "/dataScaleSize.csv", index = False)
    

In [ ]:
for core in DFCI_core_position['Core']:
    array_list = []
    core_img = DFCI_core_dict[core]
    channelNum = len(core_img)
    for channel in clusterChannels:
        m = core_img[channel]
        array_list.append(m)
    countsNoNoise=np.stack(array_list, axis=2) # count matrices in the image

    # load mask
    segMat = skimage.io.imread(f"../output/seg_results/DFCI/core_{core}/MESMER_mask.tiff")
    stats = skimage.measure.regionprops(segMat)
    labelNum = len(stats) # number of actual cells not always equal to np.max(segMat)

    # init empty containers
    data = np.zeros((labelNum,channelNum))
    dataScaleSize = np.zeros((labelNum,channelNum))
    cellSizes = np.zeros((labelNum,1))
    cell_props = np.zeros((labelNum, 3))

    # extract info
    for i in range(labelNum): # for each cell (label)
        cellLabel = stats[i].label
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i, 0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes
        cell_props[i, 0] = cellLabel
        cell_props[i, 1] = stats[i].centroid[0] # Y_cent
        cell_props[i, 2] = stats[i].centroid[1] # X_cent

    data_df = pd.DataFrame(data)
    data_df.columns = clusterChannels
    data_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), data_df), axis=1)

    dataScaleSize_df = pd.DataFrame(dataScaleSize)
    dataScaleSize_df.columns = clusterChannels
    dataScaleSize_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), dataScaleSize_df), axis = 1)

    # save all dfs
    save_dir = f'../output/extracted_info_052024/DFCI/core_{core}'

    if os.path.exists(save_dir) == False:
        os.makedirs(save_dir)

    data_full.to_csv(save_dir + "/data.csv", index = False)
    dataScaleSize_full.to_csv(save_dir + "/dataScaleSize.csv", index = False)

## Prepare image for mantis-viewer

In [ ]:
import shutil
for i in Rochester_core_position['Core']:
    seg = f"../output/seg_results/Rochester/core_{i}/MESMER_mask.tiff"
    outpath = f"../output/core_img/Rochester/core_{i}/"
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    shutil.copy2(seg, f"{outpath}MESMER_mask.tiff")
    for j in clusterChannels:
        img = Rochester_core_dict[i][j]
        tifffile.imwrite(f"../output/core_img/Rochester/core_{i}/{j}.tiff", img, photometric = 'minisblack')

In [ ]:
import shutil
for i in DFCI_core_position['Core']:
    seg = f"../output/seg_results/DFCI/core_{i}/MESMER_mask.tiff"
    outpath = f"../output/core_img/DFCI/core_{i}/"
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    shutil.copy2(seg, f"{outpath}MESMER_mask.tiff")
    for j in clusterChannels:
        img = DFCI_core_dict[i][j]
        tifffile.imwrite(f"../output/core_img/DFCI/core_{i}/{j}.tiff", img, photometric='minisblack')

### Additional processing for BCL2 and BCL6

Since the staining for BCL2 and BLC6 in the initial scan didn't have good quality. A rescan was performed to rescue the markers. Therefore, an additional round of single cell information extraction was performed to extract BCL2 and BCL6 from the rescan. 

In [ ]:
DFCI_BCL2 = tifffile.imread('../output/img_registration/fusion_to_fusion/DFCI_BCL2.tiff')
DFCI_BCL6 = tifffile.imread('../output/img_registration/fusion_to_fusion/DFCI_BCL6.tiff')

Rochester_BCL2 = tifffile.imread('../output/img_registration/fusion_to_fusion/Rochester_BCL2.tiff')
Rochester_BCL6 = tifffile.imread('../output/img_registration/fusion_to_fusion/Rochester_BCL6.tiff')

DFCI_BCL = np.stack((DFCI_BCL2, DFCI_BCL6))

Rochester_BCL = np.stack((Rochester_BCL2, Rochester_BCL6))

Visual check

In [ ]:
fig = plt.figure(figsize=(12,12))
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(DFCI_BCL[0], cmap='gray')
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(DFCI_BCL[1], cmap = 'gray')
ax3 = fig.add_subplot(2,2,3)
ax3.imshow(Rochester_BCL[0], cmap = 'gray')
ax4 = fig.add_subplot(2,2,4)
ax4.imshow(Rochester_BCL[1], cmap = 'gray')
plt.tight_layout()
plt.show()

Process the images.

In [ ]:
Rochester_BCL_dict = {}
for i in Rochester_core_position['Core']:
    x1 = Rochester_core_position[Rochester_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = Rochester_core_position[Rochester_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = Rochester_core_position[Rochester_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = Rochester_core_position[Rochester_core_position['Core'] == i]['y1_rotate'].values[0]
    core_img = Rochester_BCL[:, y1:y2, x1:x2]
    print(f"Processing core {i}")
    img_dict = process_fusion_image(core_img, '../data/MarkerList_2.txt', '../data/Rochester_config_BCL.json', coreName= i)
    Rochester_BCL_dict[i] = img_dict

DFCI_BCL_dict = {}
for i in DFCI_core_position['Core']:
    x1 = DFCI_core_position[DFCI_core_position['Core'] == i]['x2_rotate'].values[0]
    x2 = DFCI_core_position[DFCI_core_position['Core'] == i]['x1_rotate'].values[0]
    y1 = DFCI_core_position[DFCI_core_position['Core'] == i]['y2_rotate'].values[0]
    y2 = DFCI_core_position[DFCI_core_position['Core'] == i]['y1_rotate'].values[0]
    core_img = DFCI_BCL[:, y1:y2, x1:x2]
    print(f"Processing core {i}")
    img_dict = process_fusion_image(core_img, '../data/MarkerList_2.txt', '../data/DFCI_config_BCL.json', coreName= i)
    DFCI_BCL_dict[i] = img_dict

In [ ]:
txt_file = open('../data/MarkerList_2.txt', 'r')

clusterChannels = txt_file.read().split('\n')

txt_file.close()

print(clusterChannels)

Extract single cell information.

In [ ]:
for core in Rochester_core_position['Core']:
    array_list = []
    core_img = Rochester_BCL_dict[core]
    channelNum = len(core_img)
    for channel in clusterChannels:
        m = core_img[channel]
        array_list.append(m)
    countsNoNoise=np.stack(array_list, axis=2) # count matrices in the image

    # load mask
    segMat = skimage.io.imread(f"../output/seg_results/Rochester/core_{core}/MESMER_mask.tiff")
    stats = skimage.measure.regionprops(segMat)
    labelNum = len(stats) # number of actual cells not always equal to np.max(segMat)

    # init empty containers
    data = np.zeros((labelNum,channelNum))
    dataScaleSize = np.zeros((labelNum,channelNum))
    cellSizes = np.zeros((labelNum,1))
    cell_props = np.zeros((labelNum, 3))

    # extract info
    for i in range(labelNum): # for each cell (label)
        cellLabel = stats[i].label
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i, 0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes
        cell_props[i, 0] = cellLabel
        cell_props[i, 1] = stats[i].centroid[0] # Y_cent
        cell_props[i, 2] = stats[i].centroid[1] # X_cent

    data_df = pd.DataFrame(data)
    data_df.columns = clusterChannels
    data_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), data_df), axis=1)

    dataScaleSize_df = pd.DataFrame(dataScaleSize)
    dataScaleSize_df.columns = clusterChannels
    dataScaleSize_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), dataScaleSize_df), axis = 1)

    # save all dfs
    save_dir = f'../output/extracted_info/Rochester_BCL/core_{core}'

    if os.path.exists(save_dir) == False:
        os.makedirs(save_dir)

    data_full.to_csv(save_dir + "/data.csv", index = False)
    dataScaleSize_full.to_csv(save_dir + "/dataScaleSize.csv", index = False)
    
for core in DFCI_core_position['Core']:
    array_list = []
    core_img = DFCI_BCL_dict[core]
    channelNum = len(core_img)
    for channel in clusterChannels:
        m = core_img[channel]
        array_list.append(m)
    countsNoNoise=np.stack(array_list, axis=2) # count matrices in the image

    # load mask
    segMat = skimage.io.imread(f"../output/seg_results/DFCI/core_{core}/MESMER_mask.tiff")
    stats = skimage.measure.regionprops(segMat)
    labelNum = len(stats) # number of actual cells not always equal to np.max(segMat)

    # init empty containers
    data = np.zeros((labelNum,channelNum))
    dataScaleSize = np.zeros((labelNum,channelNum))
    cellSizes = np.zeros((labelNum,1))
    cell_props = np.zeros((labelNum, 3))

    # extract info
    for i in range(labelNum): # for each cell (label)
        cellLabel = stats[i].label
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i, 0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes
        cell_props[i, 0] = cellLabel
        cell_props[i, 1] = stats[i].centroid[0] # Y_cent
        cell_props[i, 2] = stats[i].centroid[1] # X_cent

    data_df = pd.DataFrame(data)
    data_df.columns = clusterChannels
    data_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), data_df), axis=1)

    dataScaleSize_df = pd.DataFrame(dataScaleSize)
    dataScaleSize_df.columns = clusterChannels
    dataScaleSize_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), dataScaleSize_df), axis = 1)

    # save all dfs
    save_dir = f'../output/extracted_info/DFCI_BCL/core_{core}'

    if os.path.exists(save_dir) == False:
        os.makedirs(save_dir)

    data_full.to_csv(save_dir + "/data.csv", index = False)
    dataScaleSize_full.to_csv(save_dir + "/dataScaleSize.csv", index = False)

### Additional processing for Myc

Myc was used for subtyping tumors. Therefore, a lower bound was also applied to it. 

In [ ]:
DFCI_geomx = tifffile.imread('../data/DFCI_geomx.ome.tiff')
DFCI_geomx_position = pd.read_csv('../data/DFCI_geomx_position.csv')
txt_file = open('../data/MarkerList_3.txt', 'r')

clusterChannels = txt_file.read().split('\n')

txt_file.close()

print(clusterChannels)
DFCI_myc_dict = {}
for i in DFCI_geomx_position['Core']:
    x1 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['x1'].values[0]
    x2 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['x2'].values[0]
    y1 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['y1'].values[0]
    y2 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['y2'].values[0]
    core_img = DFCI_geomx[:, y1:y2, x1:x2]
    print(f"Processing core {i}")
    img_dict = process_fusion_image(core_img, '../data/MarkerList_3.txt', '../data/DFCI_config_myc.json', coreName= i)
    DFCI_myc_dict[i] = img_dict

Load segmentation masks.

In [ ]:
DFCI_aligned_MESMER_mask = tifffile.imread('../output/img_registration/fusion_to_geomx/DFCI_aligned_MESMER_mask.tiff').astype('uint32')


In [ ]:
for i in tqdm(DFCI_geomx_position['Core']):
    x1 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['x1'].values[0]
    x2 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['x2'].values[0]
    y1 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['y1'].values[0]
    y2 = DFCI_geomx_position[DFCI_geomx_position['Core'] == i]['y2'].values[0]
    core_seg = DFCI_aligned_MESMER_mask[y1:y2,x1:x2]
    output_path = f"../output/seg_results/DFCI_geomx/core_{i}"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    tifffile.imwrite(f"{output_path}/MESMER_mask.tiff", core_seg)

In [ ]:
txt_file = open('../data/MarkerList_3.txt', 'r')

clusterChannels = txt_file.read().split('\n')

txt_file.close()

print(clusterChannels)

Extract single cell information

In [ ]:
for core in DFCI_geomx_position['Core']:
    array_list = []
    core_img = DFCI_myc_dict[core]
    channelNum = len(core_img)
    for channel in clusterChannels:
        m = core_img[channel]
        array_list.append(m)
    countsNoNoise=np.stack(array_list, axis=2) # count matrices in the image

    # load mask
    segMat = skimage.io.imread(f"../output/seg_results/DFCI_geomx/core_{core}/MESMER_mask.tiff")
    stats = skimage.measure.regionprops(segMat)
    labelNum = len(stats) # number of actual cells not always equal to np.max(segMat)

    # init empty containers
    data = np.zeros((labelNum,channelNum))
    dataScaleSize = np.zeros((labelNum,channelNum))
    cellSizes = np.zeros((labelNum,1))
    cell_props = np.zeros((labelNum, 3))

    # extract info
    for i in range(labelNum): # for each cell (label)
        cellLabel = stats[i].label
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i, 0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes
        cell_props[i, 0] = cellLabel
        cell_props[i, 1] = stats[i].centroid[0] # Y_cent
        cell_props[i, 2] = stats[i].centroid[1] # X_cent

    data_df = pd.DataFrame(data)
    data_df.columns = clusterChannels
    data_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), data_df), axis=1)

    dataScaleSize_df = pd.DataFrame(dataScaleSize)
    dataScaleSize_df.columns = clusterChannels
    dataScaleSize_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), dataScaleSize_df), axis = 1)

    # save all dfs
    save_dir = f'../output/extracted_info_052024/DFCI_geomx/core_{core}'

    if os.path.exists(save_dir) == False:
        os.makedirs(save_dir)

    data_full.to_csv(save_dir + "/data.csv", index = False)
    dataScaleSize_full.to_csv(save_dir + "/dataScaleSize.csv", index = False)

In [ ]:
Rochester_geomx = tifffile.imread('../data/Rochester_geomx.ome.tiff')
Rochester_geomx_position = pd.read_csv('../data/Rochester_geomx_position.csv')
Rochester_geomx_position
txt_file = open('../data/MarkerList_3.txt', 'r')

clusterChannels = txt_file.read().split('\n')

txt_file.close()

print(clusterChannels)
Rochester_myc_dict = {}
for i in Rochester_geomx_position['Core']:
    x1 = Rochester_geomx_position[Rochester_geomx_position['Core'] == i]['x1'].values[0]
    x2 = Rochester_geomx_position[Rochester_geomx_position['Core'] == i]['x2'].values[0]
    y1 = Rochester_geomx_position[Rochester_geomx_position['Core'] == i]['y1'].values[0]
    y2 = Rochester_geomx_position[Rochester_geomx_position['Core'] == i]['y2'].values[0]
    core_img = Rochester_geomx[:, y1:y2, x1:x2]
    print(f"Processing core {i}")
    img_dict = process_fusion_image(core_img, '../data/MarkerList_3.txt', '../data/Rochester_config_myc.json', coreName= i)
    Rochester_myc_dict[i] = img_dict

Extract single cell information.

In [ ]:
for core in tqdm(Rochester_geomx_position['Core']):
    array_list = []
    core_img = Rochester_myc_dict[core]
    channelNum = len(core_img)
    for channel in clusterChannels:
        m = core_img[channel]
        array_list.append(m)
    countsNoNoise=np.stack(array_list, axis=2) # count matrices in the image

    # load mask
    segMat = skimage.io.imread(f"../output/img_registration/fusion_to_geomx/Rochester/Rochester_{core}/Rochester_aligned_MESMER_mask.tiff").astype('uint32')
    stats = skimage.measure.regionprops(segMat)
    labelNum = len(stats) # number of actual cells not always equal to np.max(segMat)

    # init empty containers
    data = np.zeros((labelNum,channelNum))
    dataScaleSize = np.zeros((labelNum,channelNum))
    cellSizes = np.zeros((labelNum,1))
    cell_props = np.zeros((labelNum, 3))

    # extract info
    for i in range(labelNum): # for each cell (label)
        cellLabel = stats[i].label
        label_counts=[countsNoNoise[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i, 0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes
        cell_props[i, 0] = cellLabel
        cell_props[i, 1] = stats[i].centroid[0] # Y_cent
        cell_props[i, 2] = stats[i].centroid[1] # X_cent

    data_df = pd.DataFrame(data)
    data_df.columns = clusterChannels
    data_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), data_df), axis=1)

    dataScaleSize_df = pd.DataFrame(dataScaleSize)
    dataScaleSize_df.columns = clusterChannels
    dataScaleSize_full = pd.concat((pd.DataFrame(cell_props, columns = ["cellLabel", "Y_cent", "X_cent"]), pd.DataFrame(cellSizes, columns = ["cellSize"]), dataScaleSize_df), axis = 1)

    # save all dfs
    save_dir = f'../output/extracted_info_052024/Rochester_geomx/core_{core}'

    if os.path.exists(save_dir) == False:
        os.makedirs(save_dir)

    data_full.to_csv(save_dir + "/data.csv", index = False)
    dataScaleSize_full.to_csv(save_dir + "/dataScaleSize.csv", index = False)